In [1]:
import pandas as pd
import boto3
import os

In [2]:
s3 = boto3.resource('s3')
AWS_BUCKET_NAME = '1-cca-ted-extracted-dev'

In [7]:
# get all of the parquet files from a specific month and download them
prefix = "201901"
bucket = s3.Bucket(AWS_BUCKET_NAME)
data_path = "pq_dls"

In [8]:
downloaded_files = []
for obj in bucket.objects.filter(Prefix=prefix):
    key_to_download = obj.key
    
    # download the parquet file
    bucket.download_file(key_to_download, os.path.join(data_path, key_to_download))
    
    downloaded_files.append(key_to_download)

In [9]:
df = pd.read_parquet(os.path.join(data_path, downloaded_files[0]))
for file in downloaded_files[1:]:
    df1 = pd.read_parquet(os.path.join(data_path, file))
    df = pd.concat([df, df1], axis=0)            

In [10]:
file_name = prefix + "00_ALL.parquet"
file_path = os.path.join(data_path, file_name)
df.to_parquet(file_path)

In [11]:
# upload the new parquet file back up to S3
s3.meta.client.upload_file(Filename = file_path, Bucket = AWS_BUCKET_NAME, Key = file_name)
print("File uploaded")

# maybe delete the old individual files?
for key in downloaded_files:
    s3.Object(AWS_BUCKET_NAME, key).delete()

File uploaded


In [13]:
file_name = "20190301_2019043.parquet"

In [14]:
# try to download current month's parquet file
month_file = file_name[:6] + "00_ALL.parquet"
data_file_path = os.path.join(data_path, month_file)
try:
    bucket.download_file(month_file,data_file_path)
    
    # append current data to it
    month_df = pd.read_parquet(data_file_path)
    month_df = pd.concat([month_df, df], axis=0)
    
    # write it back to Parquet
    month_df.to_parquet(data_file_path)
except:
    # if it doesn't exist we will use the current file
    df.to_parquet(data_file_path)
    
# upload the new file to S3
s3.meta.client.upload_file(Filename = data_file_path, Bucket = AWS_BUCKET_NAME, Key = month_file)



File doesn't exist
